In [1]:
from DataHandler import DataAcquisitionHandler
from abc import abstractmethod

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
handler = DataAcquisitionHandler()
handler.load()
print(handler.get_data())

{'box_data': [{'metadata': {'start_time': 1698425150.5845294, 'length': 34.607840061187744, 'trials': [{'timestamp': (2.4168829917907715, 3.4277641773223877), 'label': True}, {'timestamp': (5.2405171394348145, 6.241200923919678), 'label': False}, {'timestamp': (8.651785373687744, 9.667745351791382), 'label': True}, {'timestamp': (11.579977989196777, 12.589327096939087), 'label': True}, {'timestamp': (14.904178380966187, 15.911416292190552), 'label': True}, {'timestamp': (18.22676968574524, 19.227490425109863), 'label': False}, {'timestamp': (21.24148988723755, 22.24232316017151), 'label': False}, {'timestamp': (24.65480375289917, 25.655165672302246), 'label': False}, {'timestamp': (27.565410375595093, 28.575204133987427), 'label': True}, {'timestamp': (30.487903833389282, 31.488284587860107), 'label': False}], 'flash_time_range': (0.1, 0.3), 'sample_time': 1, 'box_size': 'screen'}, 'data': array([[ 0.00000000e+00,  1.00000000e+00,  2.00000000e+00, ...,
         1.31000000e+02,  1.32000

In [23]:
class DataObject:


    def __init__(self, data_dict):
        keyboard_session_list = data_dict['keyboard_data']
        box_session_list = data_dict['box_data']

        self.keyboard_sessions = []
        for session_dict in keyboard_session_list:
            self.keyboard_sessions.append(SessionData(session_dict=session_dict, type='keyboard'))

        self.box_sessions = []
        for session_dict in box_session_list:
            self.box_sessions.append(SessionData(session_dict=session_dict, type='box'))


    def get_data(self, visitor=None, type='box'):
        if visitor is None:
            keyboard_data = self.keyboard_sessions
        else:
            keyboard_data = visitor.visit_data_object(object=self, type=type)
        return keyboard_data




class SessionData:
    
    def __init__(self, session_dict, type):
        metadata = session_dict['metadata']
        self.data = session_dict['data']
        self.start_time = metadata['start_time']
        self.length = metadata['length']
        self.flash_time_range = metadata['flash_time_range']
        self.sample_time = metadata['sample_time']
        self.description = metadata['description']
        self.type = type

        self.trials = []
        for trial in metadata['trials']:
            if self.type == 'keyboard':
                self.trials.append(KeyboardTrialData(trial_dict=trial, parent_session=self))
            elif self.type == 'box':
                self.trials.append(BoxTrialData(trial_dict=trial, parent_session=self))
            else:
                raise ValueError('Session type must be either keyboard or box')

    def get_data(self, visitor=None):
        if visitor is None:
            trials = self.trials
        else:
            trials = visitor.visit_session_data(session=self)
        return trials



class TrialData:

    def __init__(self, trial_dict, parent_session):
        self.timestamp = trial_dict['timestamp']
        self.label = trial_dict['label']
        self.parent_session = parent_session

    def get_data(self, visitor=None):
        if visitor is None:
            data = self.data
        else:
            data = visitor.visit_trial_data(trial=self)
        return data
    


class BoxTrialData(TrialData):

    def __init__(self, trial_dict, parent_session):
        super().__init__(trial_dict, parent_session)



class KeyboardTrialData(TrialData):

    def __init__(self, trial_dict, parent_session):
        super().__init__(trial_dict, parent_session)
        self.pattern = trial_dict['pattern']
        self.letter = trial_dict['letter']

In [80]:
class DataVisitor(object):

    @staticmethod
    def visit_data_object(object):
        raise NotImplementedError
    
    @staticmethod
    def visit_session_data(session):
        raise NotImplementedError
    
    @staticmethod
    def visit_trial_data(trial, raw_data):
        raise NotImplementedError


##########################################################################################


class MakeWindowsDataVisitor(DataVisitor):
    """
    Returns a list of tuples ( data_window, label )
    """


    @staticmethod
    def visit_data_object(object, type):
        """
        Returns a list of tuples ( data_windows, label )
        """
        sessions = object.keyboard_sessions if type == 'keyboard' else object.box_sessions

        data = []
        for session in sessions:
            data.extend(MakeWindowsDataVisitor.visit_session_data(session=session))
        return data
    

    @staticmethod
    def visit_session_data(session):
        """
        Returns a list of tuples ( data_windows, label )
        """
        data = []
        for trial in session.trials:
            data.append(MakeWindowsDataVisitor.visit_trial_data(trial=trial))
        return data
    

    @staticmethod
    def visit_trial_data(trial):
        """
        Returns a tuple ( data_windows, label )
        """
        start = trial.timestamp[0]
        end = start + trial.parent_session.sample_time

        # Make it an index for the data rather than a time
        data_len = len(trial.parent_session.data[0])
        start = (int) ( (start / trial.parent_session.length) * data_len )
        end = (int) ( (end / trial.parent_session.length) * data_len )

        return ( trial.parent_session.data[:, start:end], trial.label )

In [81]:
# Test the MakeDataWindowsVisitor

data = DataObject(handler.get_data())
formatted_data = data.get_data(visitor=MakeWindowsDataVisitor(), type='keyboard')
print(formatted_data[0][0][1])

[11476.0114522  11552.47676998 11610.2560294  11590.74295649
 11616.00042772 11654.73600086 11705.51916427 11699.7524142
 11738.06330419 11789.11468853 11834.93576466 11819.44600576
 11839.78609321 11878.81223903 11941.01714385 11984.06660367
 12067.95270061 12152.8669778  12232.88622295 12291.11251725
 12254.76858077 12148.70955333 12004.42904287 11833.03586638
 11661.88855909 11616.62627657 11652.97021305 11776.128325
 11914.01623655 12143.12161721 12376.69734677 12420.21619322
 12062.54357845 11600.30950312 11338.30235461 11327.0370754
 11364.07391597 11354.97675597 11295.92344712 11276.4550777
 11240.93815576 11183.67298647 11084.94533121 10957.33922211
 10802.37457781 10668.10764886 10620.29726747 10636.83755837
 10673.69558498 10715.26982966 10809.10245289 10934.29457358
 11069.23205486 11135.169701   11225.4707486  11317.22465959
 11426.45763474 11462.2874811  11549.43693274 11683.90502738
 11794.63556941 11735.42579835 11557.79648516 11474.69269928
 11565.19491258 11663.3414224